In [1]:
from netCDF4 import Dataset
import zarr, h5py, fsspec
import xarray as xr
import os.path
from mpi4py import MPI



In [2]:
%%script false --no-raise-error
if __name__ == '__main__':
    #Todo read up on hdf5, zarr, netcdf, xarray, mpi, lustre, ceph
    # and check back with dkrz their structure, code conventions, datasets

    # Todo setup test cases proactively to ensure proper comparability and best practice

    #Todo use netcdf dataset for testing (for now) should already bet hdf5
    dest_hdf = Dataset("data/source.nc", "w", format="NETCDF4")
    print(dest_hdf.data_model)
    dest_hdf.close()

    #Todo import netcdf data through hdf5 to zarr (for now)
    dest_zarr = zarr.open_group('data/example2.zarr', mode='w')
    zarr.copy_all(dest_hdf, dest_zarr)
    dest_zarr.tree()

    #Todo conversion to netcdf

    #for hdf5 nothing needs to be done

    #for zarr (wip). netcdf is developing its own implementation
    # but that isn't available yet so I will work on something myself in the meantime

    #Todo setup Lustre and Ceph (need info on that)

    #Todo setup benchmark (prob compression, filesize, access-time, r/w-time) for sequential access

    #Todo setup benchmark for parallel access

    #Todo setup benchmark for random access

    #Todo setup benchmark for parallel with subfileing and async / I/O


In [3]:
from keras.datasets import cifar10
import numpy as np
import h5py

(x_img_train, y_label_train), (x_img_test, y_label_test) = cifar10.load_data()

# create HDF5 file
with h5py.File('data/test_ds.hdf5', 'w') as hf:
    dset_x_train = hf.create_dataset('x_train', data=x_img_train, shape=(50000, 32, 32, 3), compression='gzip', chunks=True)
    dset_y_train = hf.create_dataset('y_train', data=y_label_train, shape=(50000, 1), compression='gzip', chunks=True)
    dset_x_test = hf.create_dataset('x_test', data=x_img_test, shape=(10000, 32, 32, 3), compression='gzip', chunks=True)
    dset_y_test = hf.create_dataset('y_test', data=y_label_test, shape=(10000, 1), compression='gzip', chunks=True)

2024-11-07 11:11:35.781298: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-07 11:11:35.781828: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-07 11:11:35.783411: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-07 11:11:35.788098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730974295.795513    5078 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730974295.79

In [4]:
#Todo prepare dataset for testing

# filepath_zarr = 'https://power-datastore.s3.amazonaws.com/v9/climatology/power_901_climatology_meteorology_utc.zarr'
# 
# filepath_mapped_zarr = fsspec.get_mapper(filepath_zarr)

ds = xr.open_dataset("data/test_ds.hdf5", engine="h5netcdf", phony_dims="access")

ds.to_netcdf('data/test_dataset.nc', mode="w")

ds.to_zarr("data/test_dataset.zarr", mode="w")

filepath_mapped_zarr = 'data/test_dataset.zarr'

filepath_nc = 'data/test_dataset.nc'



In [21]:
%%time 
ds_zarr = xr.open_zarr(filepath_mapped_zarr, consolidated=True)


CPU times: user 1.63 ms, sys: 110 μs, total: 1.74 ms
Wall time: 1.45 ms


In [22]:
%%time 
max_var = ds_zarr["x_train"].max().compute()


CPU times: user 285 ms, sys: 80 ms, total: 365 ms
Wall time: 339 ms


In [23]:
%%time
ds_hdf5 = xr.open_dataset(filename_or_obj=filepath_nc, engine="h5netcdf")


CPU times: user 6.98 ms, sys: 497 μs, total: 7.48 ms
Wall time: 6.93 ms


In [24]:
%%time
max_var = ds_hdf5["x_train"].max().compute()

CPU times: user 827 ms, sys: 1.71 ms, total: 829 ms
Wall time: 800 ms


In [4]:
import xarray as xr
ds1 = xr.open_dataset(filename_or_obj="data/4f6e0dccfcfe4f76f1e64f187cc0551c.nc", engine="netcdf4")

OSError: [Errno -101] NetCDF: HDF error: '/home/test/dkrz_dev/data/4f6e0dccfcfe4f76f1e64f187cc0551c.nc'